In [35]:
import gmpy2
import re

def findDuplicates(l):
    return list(set([x for x in l if l.count(x) > 1]))

alphabets = [str(unichr(i)) for i in range(ord('A'), ord('Z')+1)]

table = []
for i in alphabets:
    for j in alphabets:
        table.append(i + j)

P = gmpy2.next_prime(len(table))
#print P

sbox = []

affine = 37
for i in range(1,P):
    tmp = gmpy2.f_mod(gmpy2.invert(i, P) + affine, P)
    if tmp:
        sbox.append(tmp)
    else:
        sbox.append(affine)
    
#print findDuplicates(sbox)

tmp = {}
for i, j in zip(sbox, range(1, len(sbox)+1)):
    tmp[i] = j
    
isbox = []
for i in range(1, len(sbox)+1):
    isbox.append(tmp[i])
    
del(tmp)

sTable = {j: table[i-1] for i, j in zip(sbox, table)}
isTable = {j: table[i-1] for i, j in zip(isbox, table)}
#print sTable
#print isTable

def stransform(PT):
    text = [sTable[PT[2 * i] + PT[2 * i + 1]] for i in range(len(PT)/2)]
    text = [re.findall('.', i) for i in text]
    return [item for sublist in text for item in sublist]

def istransform(PT):
    text = [isTable[PT[2 * i] + PT[2 * i + 1]] for i in range(len(PT)/2)]
    text = [re.findall('.', i) for i in text]
    return [item for sublist in text for item in sublist]
    
    #return [sTable[PT[2 * i]] + sTable[PT[2 * i + 1]] for i in range(len(PT)/2)]

In [47]:
import pandas as pd

pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', None)

def _repr_latex_(self):
    return "" % self.to_latex()

pd.DataFrame._repr_latex_ = _repr_latex_  # monkey patch pandas DataFrame

def printtable(table):
    alphadict = {}
    for i in alphabets:
        alphavals = []
        for j in alphabets:
            alphavals.append(table[i + j])
        alphadict[i] = alphavals
    return alphadict

dsTable = pd.DataFrame(printtable(sTable), index=alphabets)
disTable = pd.DataFrame(printtable(isTable), index=alphabets)

In [48]:
message = "my name is abhishek"
password = "mysecret"
PT = re.findall('.', message.upper().replace(" ", ""))
K = re.findall('.', password.upper().replace(" ", ""))
print PT
print K

['M', 'Y', 'N', 'A', 'M', 'E', 'I', 'S', 'A', 'B', 'H', 'I', 'S', 'H', 'E', 'K']
['M', 'Y', 'S', 'E', 'C', 'R', 'E', 'T']


# Algorithm
## Pre Formatting
Algorithm works on even sized plain text. Thus plain text should be made even length by adding one byte pad 'P'. Further text is captalised and only alphabetic cheracters should be included.

In [49]:
message = '''If you can keep your head when all about you'''

def PreFormat(PT):
    PT0 = re.findall('.', PT.upper().replace(" ", ""))
    PT1 = []
    for i in PT0:
        if ord(i) <= ord('Z') and ord(i) >= ord('A'):
            PT1.append(i)
    if len(PT1) % 2:
        PT1.append('P')
    
    return PT1
    
PlainText = PreFormat(message)
pd.DataFrame({'PlainText': PlainText})

TypeError: not all arguments converted during string formatting

,PlainText
0,I
1,F
2,Y
3,O
4,U
5,C
6,A
7,N
8,K
9,E


## Building Blocks
Algorithm comprised of three building blocks. 
### Key Mixing
Key is expanded to the size of a plaintext by repeting it self and concatinating extra charecters. Then each charecter is added to the plaintext charecter for example. Assuming 'A' as an additive identity.

$$A + A = A$$
$$B + A = B$$
$$M + D = P$$
Similarly inverse key mixing is a cherecter substraction.
$$A - A = A$$
$$B - A = B$$
$$P - D = M$$

In [50]:
password = "mysecret"
Key = re.findall('.', password.upper().replace(" ", ""))

print 'Key:', Key

def keymixing(PT, K):
    key = [K for i in range(len(PT)/len(K)+1)]
    key = [item for sublist in key for item in sublist]
    key = key[:len(PT)]
    CT  = [str(unichr(gmpy2.f_mod(ord(i) + ord(j) - 2 * ord('A') + 1, 
                                  ord('Z') - ord('A') + 1) + ord('A'))) for i,j in zip(PT, key)]
    return CT

def invkeymixing(PT, K):
    Key = [str(unichr(ord('Z') + ord('A') - ord(i) - 1)) for i in K]
    return keymixing(PT, Key)

CipherText = keymixing(PlainText, Key)
DeCipherText = invkeymixing(CipherText, Key)

pd.DataFrame({'CipherText': CipherText, 'DeCipherText': DeCipherText}, index=PlainText)

Key: ['M', 'Y', 'S', 'E', 'C', 'R', 'E', 'T']


TypeError: not all arguments converted during string formatting

,CipherText,DeCipherText
I,V,I
F,E,F
Y,R,Y
O,T,O
U,X,U
C,U,C
A,F,A
N,H,N
K,X,K
E,D,E


### Stransform

Each pair of characters got transformed by tables given below.

While encryption Stable is used to transform pair of characters and while decryption IStable is used for inverse transform to the text.

#### Stable


In [51]:
print dsTable

    A   B   C   D   E   F   G   H   I   J   K   L   M   N   O   P   Q   R   S  \
A  BL  NY  QQ  DS  SA  AF  EB  CV  VS  GB  YG  EY  MQ  BM  FV  IP  GS  PZ  VB   
B  OL  LQ  HR  XF  JA  HN  CO  TY  JS  MI  NV  UM  PT  SU  PD  CD  LO  VN  QT   
C  KC  YT  VH  WY  OU  DO  GM  IA  YM  CE  WC  LD  GY  WG  UD  SJ  OA  IG  CH   
D  UY  HM  GN  NU  RO  FC  ZI  EX  FF  WN  RM  BD  BZ  FD  KR  GC  FO  MU  GW   
E  LV  WK  XU  RK  WX  OI  HS  QL  EV  CB  OW  FU  WP  YO  JY  BW  FZ  HK  PV   
F  FT  QW  AB  WE  YJ  SV  ME  AS  IC  IR  CM  NL  DY  RZ  ES  DL  MY  YV  BU   
G  QH  ZW  TB  SW  DT  YF  DM  KQ  HC  PO  ED  ND  UC  XL  LA  VF  BC  IV  GE   
H  LE  TA  EL  PQ  QZ  RL  HP  NI  JM  SY  DG  LB  NJ  ZS  QE  HT  QI  UX  YZ   
I  MZ  ZF  FR  RW  BQ  RI  QS  PE  EK  ZY  VQ  UO  FW  TU  NM  TM  UN  BH  DC   
J  TQ  XV  LX  XP  MP  NW  JK  DZ  ZE  IZ  HW  SO  EO  FN  VY  AO  QD  OM  OT   
K  WS  IL  DB  JP  PL  AM  HE  SR  AC  XK  BP  VM  LM  LI  TP  GK  NE  GQ  XJ   
L  QP  IY  JD  UU  NT  TO  L

#### IStable

In [52]:
print disTable

    A   B   C   D   E   F   G   H   I   J   K   L   M   N   O   P   Q   R   S  \
A  DP  HU  EW  PM  IU  YQ  ON  YB  HC  EB  UU  OG  VA  HV  QC  PY  FY  ZV  EA   
B  CF  OL  JE  CK  GA  XV  JA  TH  WO  MR  MY  LH  AV  FM  VX  WW  IW  TO  SL   
C  IK  QG  SU  SI  YX  FD  PD  IG  IF  YI  AC  GX  HY  NY  HQ  VO  GW  FU  BM   
D  BO  LD  PB  MS  KG  ND  ZP  RZ  UJ  CL  GN  LC  TR  LG  BS  OB  QJ  YE  IM   
E  KO  VR  JC  WB  YF  CX  SG  GK  YC  VG  SP  AH  GF  QK  DZ  HI  OH  XH  LW   
F  FA  PP  EY  XG  WC  ID  CM  PR  FW  ZQ  CP  OM  XZ  WS  JL  XM  NZ  YY  ST   
G  TY  GM  KQ  KH  QN  WQ  NV  BW  RC  LP  UA  VL  ZO  RM  CW  EV  VQ  MO  ZJ   
H  CR  RI  SC  FV  MV  QM  AP  GU  OR  UO  OQ  VW  VI  QT  WH  ZL  AG  JY  EX   
I  PU  MZ  UP  TG  DT  XF  QW  IY  JR  ZD  TE  NK  JB  HH  FE  UM  QH  FI  ZG   
J  NM  ZZ  MM  UK  RX  YG  AU  OP  OY  KR  FX  PW  ZH  MH  IQ  LR  GY  NP  PC   
K  AZ  SZ  ZA  ZN  II  MT  PK  RE  DO  GJ  LL  US  SV  VP  ZY  OU  WM  DE  VJ   
L  NN  AA  MN  PF  CH  QP  Y

These tables are based on multiplicative inverse in prime field $P(677)$ which is $26^2 + 1$

Further affine transform is performed in order to unco-orelate identity term 'AA' with 'AA' by adding 37.

* $'AA'$ maps to $1$ 
* $1^{-1} mod(667)= 1 $
* 1 + 37 = 38
* $38$ maps to $'BL'$

#### Security
This function is highly nonlinear function thus creates good confusion state. It increases the coorelation immunity of the cipher and also introduce local avelanche in the algorithm.

### Shift
Text is right cyclic shifted by one while encryption. And left cyclic shifted by one while decryption.

#### Security
This is a diffusion state used to disribute locally created avlanche to the next byte pair.

In [53]:
def lshift(PT):
    return PT[1:] + [PT[0]]

def rshift(PT):
    return [PT[-1]] + PT[:-1]

pd.DataFrame({'LeftShifted': lshift(PlainText), 'RightShifted': rshift(PlainText)}, index=PlainText)

TypeError: not all arguments converted during string formatting

,LeftShifted,RightShifted
I,F,P
F,Y,I
Y,O,F
O,U,Y
U,C,O
C,A,U
A,N,C
N,K,A
K,E,N
E,E,K


### Round
These building blocks are used in multiple rounds in order to encrypt a plain text.

* Rounds starts with the keymixing as other states are invertible and doesnt give any extra security.
* Key mixing is used to mix the key
* Stransform is for confusion
* shift is for diffusion
* Design is based on SP network with simpler diffusion stage and lookup table based substitution stage.
* 3 rounds are used to avoid meet in middle attack though same key is used per round (no key expansion)
* Because of two shift and three stransformation avlanche is able to diffuse between three pairs thus gives you security of 3 char pairs or 6 char or $2^{48}$
* Rounds are restricted with 3 for simplicity and easyness
* Intermediate calculations must be distroyed otherwise adversiry may use them as sidechannles information and attack the cipher.


In [54]:
def encrypt(PT, K):
    # Round one
    CT0 = keymixing(PT, K)
    CT1 = stransform(CT0)
    CT2 = lshift(CT1)    
    
    # Round two
    CT0 = keymixing(CT2, K)
    CT1 = stransform(CT0)
    CT2 = lshift(CT1)  
    
    # Round three    
    CT0 = keymixing(CT2, K)
    CT1 = stransform(CT0)
    
    CT0 = keymixing(CT1, K)
    return CT0

def decrypt(CT, K):
    # Round one
    CT0 = invkeymixing(CT, K)
    CT2 = istransform(CT0)
    
    # Round two
    CT0 = invkeymixing(CT2, K)
    CT1 = rshift(CT0)
    CT2 = istransform(CT1)
    
    # Round three
    CT0 = invkeymixing(CT2, K)
    CT1 = rshift(CT0)
    CT2 = istransform(CT1)
    
    CT0 = invkeymixing(CT2, K)
    return CT0

In [55]:
CipherText = encrypt(PlainText, Key)
DeCipherText = decrypt(CipherText, Key)

print 'Key:', Key

Key: ['M', 'Y', 'S', 'E', 'C', 'R', 'E', 'T']


In [56]:
pd.DataFrame({'CipherText': CipherText, 'DeCipherText': DeCipherText}, index=PlainText)

TypeError: not all arguments converted during string formatting

,CipherText,DeCipherText
I,M,I
F,K,F
Y,Y,Y
O,W,O
U,C,U
C,R,C
A,V,A
N,N,N
K,C,K
E,F,E
